In [1]:
import os
os.chdir('/workspace/FutureGPT2/src/')

import numpy as np
from torch import optim, nn, Tensor
from torch.nn import functional as F
import torch
import wandb
from transformers import GPT2Config, GPT2Model
import transformers
import lightning as L
from inspect import signature, _ParameterKind
import copy
import gc
import datasets
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import WandbLogger
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from itertools import repeat, product
from matplotlib import pyplot as plt
import matplotlib

from models.regression_model import *
from data.synthetic import *

%load_ext autoreload
%autoreload 2

In [2]:
fixed_data_params = {
    'seq_len': 64,
}
free_data_params = {
    'conv': 10,
    'shift': 1,
    'p': 1,
}
data_params = {**fixed_data_params, **free_data_params}

In [16]:
ds = DataLoader(
    SyntheticSeqDataset(size=50_000, **data_params), 
    batch_size=512, 
    #num_workers=20,
    drop_last=True,
)

In [6]:
ckpt = '/workspace/checkpoints/SYNTH-GPT2-MYOPIC-COS_BETA0_conv-10_shift-1_p-1_lr-0.001_n_embd-128_n_head-2_n_layer-3_activation_function-relu_global_step=58593.0_train_loss=1.24.ckpt'
model = LitGPT2RegModel.load_from_checkpoint(ckpt)

In [7]:
torch.set_grad_enabled(False)

In [17]:
loss_fn = nn.HuberLoss()
loss = 0
total = 0
for batch in tqdm(iter(ds)):
    loss += loss_fn(torch.zeros(batch['y'].shape), batch['y']).item()
    total += 1
loss /= total

  0%|          | 0/97 [00:00<?, ?it/s]

In [19]:
loss

1.2569321160463942

In [14]:
torch.var(batch['y'])

tensor(4.7274)

In [15]:
torch.mean(0.5 * batch['y']**2)

tensor(2.3649)